In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-20 00:28:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-12-20 00:28:12 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [31]:
# DELIVERABLE TWO BEGINS HERE
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on
total_votes_df = df.filter("total_votes >= 20").select(df["*"])
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15365576|R2243Y3OD8U6KQ|B00W1RTVGO|     160618734|Supertech stage l...|Musical Instruments|          5|           47|         61|   N|                N|This fills a room...|I am always looki...| 2015-08-31|
|         US|   28770559|R2TGT0CDTCAAHW|B00INJ7HBK|     157027184|Singing Machine I...|Musical Instruments| 

In [32]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_votes_df = total_votes_df.filter("(helpful_votes)/(total_votes) >= 0.5").select(total_votes_df["*"])
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15365576|R2243Y3OD8U6KQ|B00W1RTVGO|     160618734|Supertech stage l...|Musical Instruments|          5|           47|         61|   N|                N|This fills a room...|I am always looki...| 2015-08-31|
|         US|   28770559|R2TGT0CDTCAAHW|B00INJ7HBK|     157027184|Singing Machine I...|Musical Instruments| 

In [33]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_df = helpful_votes_df.filter("vine == 'Y'").select(helpful_votes_df["*"])
vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42689422|R1R9RU7JW0MFR2|B0124Y99PQ|     618027384|Casio CGP-700BK C...|Musical Instruments|          4|           20|         23|   Y|                N|Be prepared to be...|First off PLEASE ...| 2015-08-27|
|         US|   29182364|R19EFYNN3W8Q07|B00ZU4G0ZK|     499223759|TC Electronic Pol...|Musical Instruments| 

In [34]:
not_vine_df = helpful_votes_df.filter("vine == 'N'").select(helpful_votes_df["*"])
not_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15365576|R2243Y3OD8U6KQ|B00W1RTVGO|     160618734|Supertech stage l...|Musical Instruments|          5|           47|         61|   N|                N|This fills a room...|I am always looki...| 2015-08-31|
|         US|   28770559|R2TGT0CDTCAAHW|B00INJ7HBK|     157027184|Singing Machine I...|Musical Instruments| 

In [35]:
# Determining the total number of reviews from the original dataset
total_reviews = df.count()
total_reviews

904765

In [36]:
# Creating a DF that contains only the 5-star reviews from the original dataset 
five_star_reviews = df.filter("star_rating == 5").select(df["*"])
five_star_reviews.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments|          5|            0|          0|   N|                Y|          Five Stars|Nice headphones a...| 2015-08-31|
|         US|    1546619|R27HL570VNL85F|B002B55TRG|     575084461|Hohner Inc. 560BX...|Musical Instruments| 

In [37]:
# Determining the total number of 5 star reviews from the original dataset
total_five_star = five_star_reviews.count()
total_five_star

572916

In [38]:
# Creating the DF that will help me determine the percentage of total 5 star reviews that were written by Amazon Vine Members
vine_five_stars_df = five_star_reviews.filter("vine == 'Y'").select(five_star_reviews["*"])
vine_five_stars_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25575022| RCRKN2GJFD3ID|B00S2VVTP4|     489028362|Ernie Ball Regula...|Musical Instruments|          5|            0|          0|   Y|                N|Smooth Sound Quality|Smooth sound qual...| 2015-08-31|
|         US|   27769608|R1MESKI5XK08OC|B00YRKPB3K|     219663516|D'Addario Reserve...|Musical Instruments| 

In [39]:
# Counting the total amount of five star reviews that were written by Amazon Vine Members
five_star_vine_count = vine_five_stars_df.count()
five_star_vine_count

1290

In [40]:
# Creating the DF that will help me determine the percentage of total 5 star reviews that were NOT written by Amazon Vine Members
non_vine_five_stars_df = five_star_reviews.filter("vine == 'N'").select(five_star_reviews["*"])
non_vine_five_stars_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments|          5|            0|          0|   N|                Y|          Five Stars|Nice headphones a...| 2015-08-31|
|         US|    1546619|R27HL570VNL85F|B002B55TRG|     575084461|Hohner Inc. 560BX...|Musical Instruments| 

In [41]:
# Counting the total amount of five star reviews that were NOT written by Amazon Vine Members
five_star_non_vine_count = non_vine_five_stars_df.count()
five_star_non_vine_count

571626

In [42]:
# Calculating the percentage of total five star reviews that were written by Amazon Vine members
vine_percentage = (five_star_vine_count / total_five_star) * 100 
f"The percentage of total five star reviews that were written by Amazon Vine members is {vine_percentage}%."

'The percentage of total five star reviews that were written by Amazon Vine members is 0.22516389837253628%.'

In [43]:
# Calculating the percentage of total five star reviews that were NOT written by Amazon Vine members
non_vine_percentage = (five_star_non_vine_count / total_five_star) * 100 
f"The percentage of total five star reviews that were written by Amazon Vine members is {non_vine_percentage}%."

'The percentage of total five star reviews that were written by Amazon Vine members is 99.77483610162746%.'